In [9]:
import os
import pandas as pd
import dtale
from sklearn.preprocessing import MinMaxScaler

In [2]:
DATA_PATHS = [os.path.join('../data', x) for x in os.listdir('../data') if x.endswith('.csv')]
DATA_PATHS

['../data\\1010new.csv',
 '../data\\1010new_processed.csv',
 '../data\\2020new.csv',
 '../data\\2020new_processed.csv',
 '../data\\55new.csv',
 '../data\\55new_processed.csv']

In [3]:
# read csv files
path2df = {}
path2type = {}

for data_path in DATA_PATHS:
    if '_processed.csv' in data_path:
        continue
    
    set_type = 'test' if '1010new.csv' in data_path else 'train'
    df = pd.read_csv(data_path)
    
    path2df[data_path] = df
    path2type[data_path] = set_type

path2type

{'../data\\1010new.csv': 'test',
 '../data\\2020new.csv': 'train',
 '../data\\55new.csv': 'train'}

In [14]:
train_df = pd.concat([df for path, df in path2df.items() if path2type[path] == 'train'], axis=0)

scaler = MinMaxScaler()
scaler.fit(train_df.iloc[:, 2:])

train_df

,Ship_ID,Sample_Time,Latitude,Longitude,Heading,Speed,Length,Width,Work Frequency,pulse repetition frequency,pulse width
0,1,2023/12/10 10:00,16.764500,111.734640,349.713524,37.709174,160.970019,25.348867,3995.213196,590.799828,253.722855
1,2,2023/12/10 10:00,16.718144,111.911239,236.333511,44.203041,255.883095,32.097396,8802.592798,1525.520581,151.058307
2,3,2023/12/10 10:00,16.714125,112.150146,156.684916,33.478138,67.888537,16.797846,5621.453923,961.062816,120.129506
3,4,2023/12/10 10:00,16.720738,112.331162,208.730781,24.640824,13.700220,8.456157,2232.209638,329.161559,150.486240
4,5,2023/12/10 10:00,16.769878,112.553308,20.209904,46.511623,164.493393,22.591839,8880.827479,584.803285,449.489113
...,...,...,...,...,...,...,...,...,...,...,...
2995,26,2023/12/10 18:15,19.411719,113.454785,277.540434,34.908000,82.103947,16.036159,8821.358451,1543.804045,150.092424
2996,27,2023/12/10 18:15,19.868049,112.852232,283.370519,43.225632,145.920628,22.532231,8898.518969,573.302998,442.529713
2997,28,2023/12/10 18:15,19.451073,117.871343,77.573312,20.510557,21.010597,6.380410,2524.645285,400.728013,197.271288
2998,29,2023/12/10 18:15,20.093754,118.548096,67.294008,30.737095,280.221890,44.796785,4020.532591,509.895314,287.952094


In [16]:
# scaling the train and test
for data_path in DATA_PATHS:
    if '_processed.csv' in data_path:
        continue
    
    set_type = path2type[data_path]
    df = pd.read_csv(data_path)
    df.iloc[:, 2:] = scaler.transform(df.iloc[:, 2:])
    
    # handle Sample_Time to frame id 
    time2id = {v: k for k, v in list(enumerate(df['Sample_Time'].unique()))}
    df['Sample_Time'] = df['Sample_Time'].map(time2id)

    # save export the file
    df.to_csv(data_path.replace('.csv', '_processed.csv'), index=False)
    
df

,Ship_ID,Sample_Time,Latitude,Longitude,Heading,Speed,Length,Width,Work Frequency,pulse repetition frequency,pulse width
0,1,0,0.577055,0.766063,0.494135,0.231093,0.038894,0.103996,0.087730,0.085369,0.210773
1,2,0,0.589356,0.766207,0.013068,0.926630,0.464641,0.362710,0.995950,0.241873,0.680213
2,3,0,0.594323,0.766937,0.058514,0.681222,0.926641,0.995562,0.471371,0.230521,0.668412
3,4,0,0.592847,0.765532,0.527440,0.936223,0.998199,0.732907,0.954327,0.261254,0.956397
4,5,0,0.600468,0.766558,0.459925,0.326394,0.110029,0.377957,0.165327,0.182099,0.357231
...,...,...,...,...,...,...,...,...,...,...,...
2995,26,99,0.623165,0.496304,0.784617,0.522430,0.227904,0.286188,0.988089,0.983479,0.068716
2996,27,99,0.676968,0.436715,0.801049,0.798498,0.432459,0.418812,0.999089,0.226338,0.694949
2997,28,99,0.627805,0.933073,0.221024,0.044567,0.032079,0.089055,0.090467,0.091703,0.169746
2998,29,99,0.703579,1.000000,0.192052,0.383994,0.862940,0.873366,0.303712,0.176870,0.363932


In [ ]:
# # Assigning a reference to a running D-Tale process
# df = pd.read_csv(DATA_PATHS[0])
# df_ = pd.read_csv(DATA_PATHS[2])
# d = dtale.show(df)
# d.open_browser()

# d_ = dtale.show(df_)
# d_.open_browser()